In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pytorch_lightning import seed_everything, LightningModule, Trainer
from sklearn.utils import class_weight
import torch.nn as nn
import torch
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint,LearningRateMonitor
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau,CosineAnnealingWarmRestarts,OneCycleLR,CosineAnnealingLR
import torchvision
from sklearn.metrics import classification_report,f1_score,accuracy_score
from PIL import Image
from torch.utils.data import DataLoader, Dataset,random_split
import timm
import torchmetrics
import torchvision.models as models

In [2]:
img_size=224
import albumentations as A
from albumentations.pytorch import ToTensorV2
aug= A.Compose([
            A.Resize(img_size,img_size),
            #A.RandomCrop(img_size,img_size),
            A.HorizontalFlip(0.5),
            #A.ShiftScaleRotate(rotate_limit=3),
            #A.CoarseDropout(8,64,64),
            A.Normalize(),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [3]:
class DataReader(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        x=self.dataset[index][0]
        y=self.dataset[index][1]
        if self.transform:
            x=np.array(x)
            x=self.transform(image=x)['image']
        return x, y
    
    def __len__(self):
        return len(self.dataset)

In [4]:
class OurModel(LightningModule):
    def __init__(self):
        super(OurModel,self).__init__()
        #architecute
        self.model =  timm.create_model(model_name,pretrained=True)
        self.fc1=nn.Linear(1000,500)
        self.relu=nn.ReLU()
        self.fc2= nn.Linear(500,5)
 
        #parameters
        self.lr=1e-3
        self.batch_size=72
        self.numworker=12
        self.acc = torchmetrics.Accuracy()
        self.criterion=nn.CrossEntropyLoss()
        #LabelSmoothing()
        
        self.trainacc,self.valacc=[],[]
        self.trainloss,self.valloss=[],[]
        
        self.scheduler='constant'
        
        self.dataset=torchvision.datasets.ImageFolder('dataset')
        
        self.train_set, self.val_set =random_split(self.dataset, 
                            [int(len(self.dataset)*0.8), int(len(self.dataset)*0.2)])
    def forward(self,x):
        x= self.model(x)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x

    def configure_optimizers(self):
        opt=torch.optim.AdamW(params=self.parameters(),lr=self.lr )
        if self.scheduler=='cosine':
            scheduler=CosineAnnealingLR(opt,T_max=10,  eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='reduce':
            scheduler=ReduceLROnPlateau(opt,mode='min', factor=0.5, patience=5)
            return {'optimizer': opt,'lr_scheduler':scheduler,'monitor':'val_loss'}
        elif self.scheduler=='warm':
            scheduler=CosineAnnealingWarmRestarts(opt,T_0=3, T_mult=1, eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='constant':
            return opt
 
    def train_dataloader(self):
        return DataLoader(DataReader(self.train_set,aug), batch_size = self.batch_size, 
                          num_workers=self.numworker,
                          pin_memory=True,shuffle=False)

    def training_step(self,batch,batch_idx):
        image,label=batch
        out = self(image)
        loss=self.criterion(out,label)
        acc=self.acc(out,label)
        return {'loss':loss,'acc':acc}

    def training_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        acc=torch.stack([x["acc"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        self.trainacc.append(acc)
        self.trainloss.append(loss)
        #print('training loss accuracy ',self.current_epoch,loss, acc)
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        
    def val_dataloader(self):
        ds=DataLoader(DataReader(self.val_set,aug), batch_size = self.batch_size,
                      num_workers=self.numworker,pin_memory=True, shuffle=False)
        return ds

    def validation_step(self,batch,batch_idx):
        image,label=batch
        out=self(image)
        loss=self.criterion(out,label)
        acc=self.acc(out,label)
        return {'loss':loss,'acc':acc}

    def validation_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        acc=torch.stack([x["acc"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        self.valacc.append(acc)
        self.valloss.append(loss)
        print('validation loss accuracy ',self.current_epoch,loss, acc)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

In [5]:
model_name='resnest26d'
model=OurModel()

In [6]:
lr_monitor = LearningRateMonitor(logging_interval='epoch')
trainer = Trainer(max_epochs=10, auto_lr_find=False, auto_scale_batch_size=False,
                deterministic=True,
                gpus=-1,precision=16,
                accumulate_grad_batches=4,
                stochastic_weight_avg=False,
                enable_progress_bar = False,
                num_sanity_val_steps=0,
                callbacks=[lr_monitor]
                 )

Using 16bit native Automatic Mixed Precision (AMP)
[W Context.cpp:69] Warning: torch.set_deterministic is in beta, and its design and  functionality may change in the future. (function operator())
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [7]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2021-12-06 20:32:28.719403: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 27.5 M
1 | fc1       | Linear           | 500 K 
2 | relu      | ReLU             | 0     
3 | fc2       | Linear           | 2.5 K 
4 | acc       | Accuracy         | 0     
5 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
28.0 M    Trainable params
0         Non-trainable params
28.0 M    Total params
55.972    Total estimated model params size (MB)


validation loss accuracy  0 1.32 0.45
validation loss accuracy  1 0.79 0.68
validation loss accuracy  2 0.73 0.69
validation loss accuracy  3 0.86 0.66
validation loss accuracy  4 0.95 0.62
validation loss accuracy  5 0.73 0.71
validation loss accuracy  6 1.09 0.58
validation loss accuracy  7 0.83 0.68
validation loss accuracy  8 0.8 0.69
validation loss accuracy  9 0.86 0.67


In [8]:
trainer.validate(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


validation loss accuracy  9 0.86 0.67
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_acc': 0.6700000166893005, 'val_loss': 0.8600000143051147}
--------------------------------------------------------------------------------


[{'val_loss': 0.8600000143051147, 'val_acc': 0.6700000166893005}]

In [15]:
loader=model.val_dataloader()
model.cuda().eval()
labels,preds=[],[]
with torch.no_grad():
    for batch in loader:
        image,label=batch
        pred=model(image.cuda())
        pred=torch.argmax(pred,dim=1).detach().cpu().numpy()
        labels.append(label.cpu().numpy())
        preds.append(pred)

In [22]:
from sklearn.metrics import classification_report
print(classification_report(np.hstack(labels),np.hstack(preds)))

              precision    recall  f1-score   support

           0       0.63      0.93      0.75       704
           1       0.57      0.74      0.65       584
           2       0.69      0.55      0.61       571
           3       0.71      0.51      0.60       627
           4       0.89      0.58      0.70       614

    accuracy                           0.67      3100
   macro avg       0.70      0.66      0.66      3100
weighted avg       0.70      0.67      0.67      3100

